# 공개 이미지 데이터셋 MinIO 업로드 스크립트

zip 파일로 된 이미지 데이터셋을 풀어서 MinIO 서버에 업로드합니다.

In [ ]:
import os
from pathlib import Path
import sys
import zipfile
import tempfile
import shutil

# src 디렉토리를 파이썬 경로에 추가
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from data_engineering.obj_storage.boto3 import MinIOClient

In [ ]:
# MinIO 클라이언트 초기화
minio_client = MinIOClient()

# 버킷 이름 설정
BUCKET_NAME = "tada"  # 원하는 버킷 이름으로 변경하세요

# 데이터셋 경로 설정
DATASET_PATH = project_root / "temp" / / "239.건축 도면 데이터" / "01-1.정식개방데이터" / "Training" / "01.원천데이터"

In [ ]:
# 버킷이 없으면 생성
if not minio_client.bucket_exists(BUCKET_NAME):
    minio_client.create_bucket(BUCKET_NAME)
    print(f"버킷 '{BUCKET_NAME}' 생성 완료")
else:
    print(f"버킷 '{BUCKET_NAME}' 이미 존재함")

In [ ]:
# 지원하는 이미지 확장자
IMAGE_EXTENSIONS = {".jpg", ".jpeg", ".png", ".gif", ".bmp"}

def is_image_file(filename):
    return any(filename.lower().endswith(ext) for ext in IMAGE_EXTENSIONS)

# 임시 디렉토리 생성
temp_dir = Path(tempfile.mkdtemp())
print(f"임시 디렉토리 생성됨: {temp_dir}")

try:
    from tqdm.notebook import tqdm
    
    # ZIP 파일 목록 가져오기
    zip_files = list(DATASET_PATH.glob("*.zip"))
    print(f"처리할 ZIP 파일 수: {len(zip_files)}")
    
    # 각 ZIP 파일 처리
    for zip_path in tqdm(zip_files, desc="ZIP 파일 처리 중"):
        category = zip_path.stem  # ZIP 파일 이름을 카테고리로 사용
        print(f"\n처리 중인 카테고리: {category}")
        
        # ZIP 파일 압축 해제
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # 이미지 파일만 추출
            for file_info in tqdm(zip_ref.filelist, desc="파일 추출 중"):
                if is_image_file(file_info.filename):
                    # 파일 경로에서 디렉토리 구조 유지
                    relative_path = Path(file_info.filename)
                    object_key = f"{category}/{relative_path}"
                    
                    # 임시 파일로 추출
                    temp_path = temp_dir / relative_path.name
                    with zip_ref.open(file_info) as source, open(temp_path, 'wb') as target:
                        shutil.copyfileobj(source, target)
                    
                    try:
                        # MinIO에 업로드
                        minio_client.upload_file(
                            bucket_name=BUCKET_NAME,
                            object_name=object_key,
                            file_path=str(temp_path)
                        )
                    except Exception as e:
                        print(f"업로드 실패: {object_key} - {str(e)}")
                    
                    # 임시 파일 삭제
                    temp_path.unlink()

finally:
    # 임시 디렉토리 정리
    shutil.rmtree(temp_dir)
    print("임시 디렉토리 정리 완료")

In [ ]:
# 업로드된 파일 확인
uploaded_files = minio_client.list_objects(BUCKET_NAME)
print(f"업로드된 총 파일 수: {len(uploaded_files)}")

# 처음 몇 개의 파일 출력
print("\n업로드된 파일 목록 (처음 5개):")
for obj in list(uploaded_files)[:5]:
    print(f"- {obj}")